In [ ]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 fosforio fosforml numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays faker
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15

In [1]:
from fosforio import snowflake
from fosforio import get_dataframe
from datetime import datetime, timedelta

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


In [2]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import seaborn as sns

Matplotlib created a temporary cache directory at /tmp/matplotlib-__8e11_z because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [ ]:
snowflake.get_connection(connection_name="TTH_REV_OPT_CXN")
data = get_dataframe("BOOKINGS")
data

In [3]:
data = pd.read_csv('./bookings.csv')
data

,HOTEL,IS_CANCELED,LEAD_TIME,ARRIVAL_DATE_YEAR,MONTH,ARRIVAL_DATE_WEEK_NUMBER,ARRIVAL_DATE_DAY_OF_MONTH,EXPECTED_ARRIVAL_DATE,RESERVATION_STATUS,RESERVATION_STATUS_DATE,TOTAL_STAY_NIGHTS,TALLY_DAYS,STAYS_IN_WEEKEND_NIGHTS,STAYS_IN_WEEK_NIGHTS,ADULTS,CHILDREN,BABIES,TOTAL_GUESTS,AVG_ROOMS_PER_NIGHT,TOTAL_ROOM_NIGHTS,MEAL,COUNTRY,MARKET_SEGMENT,DISTRIBUTION_CHANNEL,PREVIOUS_CANCELLATIONS,PREVIOUS_BOOKINGS_NOT_CANCELED,RESERVED_ROOM_TYPE,ASSIGNED_ROOM_TYPE,DEPOSIT_TYPE,DAYS_IN_WAITING_LIST,CUSTOMER_TYPE,ADR
0,City Hotel,1,272,2021,7,29,16,2021-07-16,Canceled,17-10-2020,2,0.272,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
1,City Hotel,1,307,2021,8,34,20,2021-08-20,Canceled,17-10-2020,2,0.307,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
2,City Hotel,1,258,2021,7,27,2,2021-07-02,Canceled,17-10-2020,2,0.258,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
3,City Hotel,1,307,2021,8,34,20,2021-08-20,Canceled,17-10-2020,2,0.307,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
4,City Hotel,1,279,2021,7,30,23,2021-07-23,Canceled,17-10-2020,2,0.279,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117100,Resort Hotel,0,269,2023,8,34,24,2023-08-24,Check.Out,10-09-2023,17,17.000,4,13,2,0,0,2,1,17,BB,GBR,Offline TA/TO,TA/TO,0,0,D,D,No Deposit,0,Contract,84.80
117101,Resort Hotel,0,212,2023,8,35,31,2023-08-31,Check.Out,10-09-2023,10,10.000,2,8,2,1,0,3,1,10,BB,GBR,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,89.75
117102,Resort Hotel,0,204,2023,8,35,29,2023-08-29,Check.Out,12-09-2023,14,14.000,4,10,2,0,0,2,1,14,BB,IRL,Direct,Direct,0,0,E,E,No Deposit,0,Transient,153.57
117103,Resort Hotel,0,161,2023,8,35,31,2023-08-31,Check.Out,14-09-2023,14,14.000,4,10,2,0,0,2,1,14,HB,DEU,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,99.06


In [ ]:
data.tail()

In [4]:
data['EXPECTED_ARRIVAL_DATE'] = pd.to_datetime(data['EXPECTED_ARRIVAL_DATE'], errors='coerce')
data['RESERVATION_STATUS_DATE'] = pd.to_datetime(data['RESERVATION_STATUS_DATE'], errors='coerce')
data['EXPECTED_ARRIVAL_DATE'] - data['RESERVATION_STATUS_DATE']

<ipython-input-4-48c2fc89ab93>:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['RESERVATION_STATUS_DATE'] = pd.to_datetime(data['RESERVATION_STATUS_DATE'], errors='coerce')


0        272 days
1        307 days
2        258 days
3        307 days
4        279 days
           ...   
117100   -17 days
117101   -10 days
117102   -14 days
117103   -14 days
117104   -14 days
Length: 117105, dtype: timedelta64[ns]

In [5]:
data['LEAD_TIME'].min()

0

In [6]:
data['LEAD_TIME'].max()

709

In [7]:
data['LEAD_TIME'].mean()

105.19527774219718

In [8]:
data.columns

Index(['HOTEL', 'IS_CANCELED', 'LEAD_TIME', 'ARRIVAL_DATE_YEAR', 'MONTH',
       'ARRIVAL_DATE_WEEK_NUMBER', 'ARRIVAL_DATE_DAY_OF_MONTH',
       'EXPECTED_ARRIVAL_DATE', 'RESERVATION_STATUS',
       'RESERVATION_STATUS_DATE', 'TOTAL_STAY_NIGHTS', 'TALLY_DAYS',
       'STAYS_IN_WEEKEND_NIGHTS', 'STAYS_IN_WEEK_NIGHTS', 'ADULTS', 'CHILDREN',
       'BABIES', 'TOTAL_GUESTS', 'AVG_ROOMS_PER_NIGHT', 'TOTAL_ROOM_NIGHTS',
       'MEAL', 'COUNTRY', 'MARKET_SEGMENT', 'DISTRIBUTION_CHANNEL',
       'PREVIOUS_CANCELLATIONS', 'PREVIOUS_BOOKINGS_NOT_CANCELED',
       'RESERVED_ROOM_TYPE', 'ASSIGNED_ROOM_TYPE', 'DEPOSIT_TYPE',
       'DAYS_IN_WAITING_LIST', 'CUSTOMER_TYPE', 'ADR'],
      dtype='object')

In [9]:
pd.to_datetime(data['EXPECTED_ARRIVAL_DATE']).min()

Timestamp('2021-07-01 00:00:00')

In [10]:
pd.to_datetime(data['EXPECTED_ARRIVAL_DATE']).max()

Timestamp('2023-08-31 00:00:00')

In [11]:
data['EXPECTED_ARRIVAL_DATE'] = pd.to_datetime(data['EXPECTED_ARRIVAL_DATE'], errors='coerce')
data['RESERVATION_STATUS_DATE'] = pd.to_datetime(data['RESERVATION_STATUS_DATE'], errors='coerce')

In [12]:
data.sort_values(by='EXPECTED_ARRIVAL_DATE')['EXPECTED_ARRIVAL_DATE']

2819    2021-07-01
2953    2021-07-01
1427    2021-07-01
2950    2021-07-01
2949    2021-07-01
           ...    
34089          NaT
34149          NaT
34275          NaT
34319          NaT
34859          NaT
Name: EXPECTED_ARRIVAL_DATE, Length: 117105, dtype: datetime64[ns]

In [13]:
original_last_date = datetime(2023, 8, 31)
target_date = datetime(2024, 6, 30)

In [14]:
original_min_date = data['EXPECTED_ARRIVAL_DATE'].min()
original_min_date

Timestamp('2021-07-01 00:00:00')

In [15]:
days_to_add = (target_date - original_last_date).days
days_to_add

304

In [16]:
data['NEW_RES_TIME'] = 0

In [18]:
data['NEW_RES_TIME'].loc[0]

0

In [19]:
data['LEAD_TIME'].iloc[0]

272

In [20]:
int(data['LEAD_TIME'].iloc[0])

272

In [ ]:
data['EXPECTED_ARRIVAL_DATE'].iloc[0] -  timedelta(days=int(data['LEAD_TIME'].iloc[0]))

In [21]:
for index, row in data.iterrows():
    data['NEW_RES_TIME'].iloc[index] = data['EXPECTED_ARRIVAL_DATE'].iloc[index] - timedelta(days=int(data['LEAD_TIME'].iloc[index]))

<ipython-input-21-9203a18b5794>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['NEW_RES_TIME'].iloc[index] = data['EXPECTED_ARRIVAL_DATE'].iloc[index] - timedelta(days=int(data['LEAD_TIME'].iloc[index]))


In [22]:
data['NEW_RES_TIME']

0         2020-10-17 00:00:00
1         2020-10-17 00:00:00
2         2020-10-17 00:00:00
3         2020-10-17 00:00:00
4         2020-10-17 00:00:00
                 ...         
117100    2022-11-28 00:00:00
117101    2023-01-31 00:00:00
117102    2023-02-06 00:00:00
117103    2023-03-23 00:00:00
117104    2023-02-01 00:00:00
Name: NEW_RES_TIME, Length: 117105, dtype: object

In [23]:
data['EXPECTED_ARRIVAL_DATE'] = data['EXPECTED_ARRIVAL_DATE'] + timedelta(days=days_to_add)
data['RESERVATION_STATUS_DATE'] = data['NEW_RES_TIME'] + timedelta(days=days_to_add)

In [24]:
data['LEAD_TIME_UPDATED'] = data['EXPECTED_ARRIVAL_DATE'] - data['RESERVATION_STATUS_DATE']

<ipython-input-24-23265ab83679>:1: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  data['LEAD_TIME_UPDATED'] = data['EXPECTED_ARRIVAL_DATE'] - data['RESERVATION_STATUS_DATE']


In [32]:
data['LEAD_TIME_UPDATED'].apply(lambda x: int(x.split()[0]))

AttributeError: 'Timedelta' object has no attribute 'split'

In [ ]:
data['LEAD_TIME_UPDATED'].max()

In [ ]:
data['LEAD_TIME_UPDATED'] = data['LEAD_TIME_UPDATED'].dt.days

In [ ]:
data['LEAD_TIME_UPDATED']

In [ ]:
data[data["LEAD_TIME_UPDATED"] < 0]

In [ ]:
data['EXPECTED_ARRIVAL_DATE'].min()

In [ ]:
data['EXPECTED_ARRIVAL_DATE'].max()

In [ ]:
import os
from snowflake.snowpark.session import Session
user = os.getenv("user")
warehouse = os.getenv("warehouse")
schema= os.getenv("schema")
database = os.getenv("database")
role =  os.getenv("role")
account =  os.getenv("account")
password= os.getenv("password")

connection_params = dict(user=user, 
                         password=password, 
                         account=account, 
                         warehouse=warehouse, 
                         database=database,
                         schema=schema, 
                         role=role)

session = Session.builder.configs(connection_params).create()

session.sql('use warehouse {};'.format(warehouse)).collect()

session.sql('use database {};'.format(database)).collect()

session.sql('use schema {}.{};'.format(database, schema)).collect()

In [ ]:
data.info()

In [ ]:
data["EXPECTED_ARRIVAL_DATE"] = pd.to_datetime(data["EXPECTED_ARRIVAL_DATE"]).dt.strftime("%Y-%m-%d %H:%M:%S")


In [ ]:
data.info()

In [ ]:
df_model=session.createDataFrame(
        data_backup.values.tolist(),
        schema=data_backup.columns.tolist())
df_model.write.mode("overwrite").save_as_table("TTH_DB.TTH_REV_OPT_Schema.BOOKINGS")

In [ ]:
data['EXPECTED_ARRIVAL_DATE'].isna().sum()

In [ ]:
data_backup = data.dropna(axis=0)

In [ ]:
data_backup['EXPECTED_ARRIVAL_DATE'].min()

In [ ]:
data_backup['EXPECTED_ARRIVAL_DATE'].max()

In [ ]:
117105 - 117009